In [23]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer


In [24]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/newdata/member_sample.csv


In [25]:
# member_sample.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
data = pd.read_csv('/kaggle/input/newdata/member_sample.csv', delimiter=',')
data.dataframeName = 'member_sample.csv'
nRow, nCol = data.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 21344 rows and 47 columns


Let's take a quick look at what the data looks like:

In [26]:
data.head(5)

,Household_Key,ZIP5,FSV_CMSI_Flag,FSV_Credit_Card_Flag,FSV_Deposit_Program_Flag,FSV_Home_Equity_Flag,FSV_ID_Theft_Flag,FSV_Mortgage_Flag,INS_Client_Flag,TRV_Globalware_Flag,...,Member_Match_Flag,Motorcycle_Indicator,Plus_Cost,Premier_Cost,Prob1_Code_Description,SC_Vehicle_Manufacturer_Name,SC_Vehicle_Model_Name,Total_Cost,Was_Duplicated,Was_Towed_To_AAR_Referral
0,10462590,6511,N,N,N,N,N,N,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4500791,2893,N,Y,N,N,N,N,N,N,...,1.0,N,0.0,0.0,Tow,TOYOTA,CAMRY,32.5,0.0,1.0
2,4500791,2893,N,Y,N,N,N,N,N,N,...,1.0,N,0.0,0.0,Battery Service,TOYOTA,CAMRY,30.0,0.0,0.0
3,4500791,2893,N,Y,N,N,N,N,N,N,...,1.0,N,0.0,0.0,Battery Service,TOYOTA,CAMRY,32.5,0.0,0.0
4,4500791,2893,N,Y,N,N,N,N,N,N,...,1.0,N,0.0,0.0,Jump Start,TOYOTA,CAMRY,30.0,0.0,0.0


In [27]:
data.describe()

,Household_Key,ZIP5,Length_Of_Residence,Do_Not_Direct_Mail_Solicit,Email_Available,ERS_ENT_Count_Year_1,ERS_ENT_Count_Year_2,ERS_ENT_Count_Year_3,ERS_Member_Cost_Year_1,ERS_Member_Cost_Year_2,...,Basic_Cost,Calculated_Tow_Miles,Is_Duplicate,Is_NSR,Member_Match_Flag,Plus_Cost,Premier_Cost,Total_Cost,Was_Duplicated,Was_Towed_To_AAR_Referral
count,2.134400e+04,21344.000000,12010.000000,21343.000000,21343.000000,16675.000000,16675.000000,16675.000000,16675.000000,16675.000000,...,13944.000000,13861.000000,13997.0,13997.000000,13997.0,13944.000000,13911.000000,13944.000000,13997.000000,13997.000000
mean,1.597040e+07,2944.899831,11.524979,0.052851,0.536101,0.518561,0.936252,0.959760,28.068167,50.970628,...,42.629593,2.379071,0.0,0.059156,1.0,3.450947,0.006901,47.273696,0.065157,0.013646
std,2.138079e+07,478.757704,5.495280,0.223741,0.498707,0.985532,1.253213,1.322932,57.576978,78.213543,...,15.713987,7.030493,0.0,0.235924,0.0,18.213483,0.692848,86.869110,0.246811,0.116020
min,8.750000e+02,2801.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.350387e+06,2857.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,30.000000,0.000000,0.0,0.000000,1.0,0.000000,0.000000,30.500000,0.000000,0.000000
50%,9.811235e+06,2888.000000,15.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,53.000000,0.000000,0.0,0.000000,1.0,0.000000,0.000000,53.000000,0.000000,0.000000
75%,1.654012e+07,2910.000000,15.000000,0.000000,1.000000,1.000000,2.000000,1.000000,36.000000,66.750000,...,53.000000,1.000000,0.0,0.000000,1.0,0.000000,0.000000,53.000000,0.000000,0.000000
max,1.000791e+08,6902.000000,49.000000,1.000000,1.000000,7.000000,10.000000,9.000000,560.740000,904.390000,...,106.390000,142.000000,0.0,1.000000,1.0,530.000000,81.000000,9869.000000,1.000000,1.000000


In [34]:
data_encoded = pd.get_dummies(data)

In [ ]:
new_data = data.copy()
cols_with_missing = (col for col in new_data.columns 
                                 if new_data[col].isnull().any())
for col in cols_with_missing:
    new_data[col + '_was_missing'] = new_data[col].isnull()
    my_imputer = SimpleImputer()
new_data = pd.DataFrame(my_imputer.fit_transform(new_data))
new_data.columns = data.columns

In [35]:
data_encoded.iloc[:,5:].head(5)

,ERS_ENT_Count_Year_1,ERS_ENT_Count_Year_2,ERS_ENT_Count_Year_3,ERS_Member_Cost_Year_1,ERS_Member_Cost_Year_2,ERS_Member_Cost_Year_3,Member_Tenure_Years,Basic_Cost,Calculated_Tow_Miles,Is_Duplicate,...,SC_Vehicle_Model_Name_YARIS,SC_Vehicle_Model_Name_YUKON,SC_Vehicle_Model_Name_YUKON XL,SC_Vehicle_Model_Name_YUKON XL 1500,SC_Vehicle_Model_Name_YUKON XL 2500,SC_Vehicle_Model_Name_Yukon XL 1500,SC_Vehicle_Model_Name_Z3,SC_Vehicle_Model_Name_Z4,SC_Vehicle_Model_Name_ZEPHYR,SC_Vehicle_Model_Name_murano
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,2.0,0.0,0.0,65.0,49.0,32.5,0.8,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,2.0,0.0,0.0,65.0,49.0,30.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,2.0,0.0,0.0,65.0,49.0,32.5,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,2.0,0.0,0.0,65.0,49.0,30.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
labels = np.array(data_encoded['Total_Cost'])

In [12]:
data_encoded= data_encoded.drop('Total_Cost', axis = 1)

In [13]:
feature_list = list(data_encoded.columns)

In [14]:
features = np.array(data_encoded)

In [15]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [16]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (16008, 1462)
Training Labels Shape: (16008,)
Testing Features Shape: (5336, 1462)
Testing Labels Shape: (5336,)


In [17]:
baseline_preds = test_features[:, feature_list.index('Premier_Cost')]

In [18]:
baseline_errors = abs(baseline_preds - test_labels)

In [19]:
print('Average baseline error: ', round(np.mean(baseline_errors), 2))

Average baseline error:  nan


In [20]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [21]:
rf.fit(train_features, train_labels);

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').